In [1]:
from bs4 import BeautifulSoup
import re
import requests
import urllib
import ctypes
from datetime import timedelta, date
from proxy_requests import ProxyRequests
import time
import random
import json

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
def prepend(x):
    x = str(x)
    if len(x) == 2:
        return x
    else:
        return '0' + x
        
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',}

def get_soup(url):
    for i in range(60):
        r = requests.get(url, headers=headers, timeout=15, proxies = urllib.request.getproxies())

        if r.status_code != 200:
            print('Request status code:\t' + str(r.status_code))
            time.sleep(60)
            continue

        s = BeautifulSoup(r.content.decode(), "html.parser")

        if len(s.get_text()) < 100:
            print('Soup error...')
            time.sleep(60)            
            continue
        
        return s
    print('OH NO!!!')


In [8]:
start_date = date(2020, 4, 1)
end_date = date(2020, 4, 25)

In [ ]:
for single_date in daterange(start_date, end_date):
    print('Parsing', single_date)
    suff = f'{single_date.year}/{prepend(single_date.month)}/{prepend(single_date.day)}'
    init_url = f'https://lenta.ru/news/{suff}'
    
    s = get_soup(init_url)
    
    for x in s.find_all('a', href=True):
        href = x.get('href')
        if href.startswith(f'/news/{suff}'):
            news_url = f'https://lenta.ru{href}'
            
            news_soup = get_soup(news_url)
            
            print('Parsing', news_url)
            
            title = news_soup.find('h1', attrs={'class':'b-topic__title'}).text
            body = news_soup.find_all('div', attrs={'class':'b-text clearfix js-topic__text'})[0]
            text = ' '.join([el.text for el in body.find_all('p')])
            
            with open('lenta_parsed_data.jsonl', 'a', encoding='utf-8') as f:
                json.dump({
                    'title': title,
                    'text': text,
                    'date': str(single_date)
                }, f)
                f.write('\n')
            
            time.sleep(15 + random.randint(1, 30))

Parsing 2020-04-01
Parsing https://lenta.ru/news/2020/04/01/pension/


In [11]:
!wc -l lenta_parsed_data.jsonl

377 lenta_parsed_data.jsonl
